In [1]:
import subprocess
import glob
import os
from os.path import exists
import re
import pandas as pd
from multiprocessing import Pool
# import pyreadr

#### the genome, individual bams and so on have been prepared in the previous diffTF, mainly addiing the prefix "chr"


### add "chr" prefix to chromosome names in GTF

In [5]:
! awk '!/^#/ {$1 = "chr"$1} {print}' /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.gtf > ./TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_modified.gtf

In [14]:
! grep -v '^#' /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_modified.gtf > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_no_header.gtf

In [17]:
! awk 'BEGIN{OFS="\t"} {$1=$1; print}' /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_no_header.gtf > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_no_header_tabbed.gtf


#### merge bams

In [2]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C12*.bam

In [3]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C14_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C1*.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C2*.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C3*.bam

In [4]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/DM*.bam

In [5]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/OK*.bam

In [6]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C14_merged.bam 

In [7]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam

In [11]:
def index_bam(bam_file):
    try:
        print(f"Indexing {bam_file}...")
        subprocess.run(["samtools", "index",
                        '-@', '30',
                        bam_file], check=True)
        print(f"Finished indexing {bam_file}.")
    except subprocess.CalledProcessError as e:
        print(f"Error indexing {bam_file}: {e}")

# Path to the directory containing your BAM files
directory = '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam'

# List of all BAM files in the directory
bam_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.bam')]

# Number of worker processes to use
num_workers = 3

# Create a pool of worker processes
with Pool(num_workers) as pool:
    # Map the index_bam function to each BAM file
    pool.map(index_bam, bam_files)

Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam...Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam...Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam...


Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam.
Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam...
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam.
Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam...
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam.
Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C14_merged.bam...
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam.
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam.
Fin

#### TOBIAS use broad peaks, refer to 1_DM1_DM2d_ATAC_call_peak.ipynb and https://github.com/loosolab/TOBIAS/wiki/test-data

In [49]:
! cat /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/C12A_clean_filtered_peaks.broadPeak /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/C12B_clean_filtered_peaks.broadPeak /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/C12C_clean_filtered_peaks.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_broad_peaks.bed

In [50]:
! cat /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/DM*.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_broad_peaks.bed

In [51]:
! cat /media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/12_macs2peaks_broad/OK*.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM6_DM7_broad_peaks.bed

In [52]:
! cat /media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/12_macs2peaks_broad/C*.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C14_broad_peaks.bed

#### further merge bed peaks files by DM or Col

In [53]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C*.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_C14_merged_peaks.bed

In [54]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/D*.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_DM6_DM7_merged_peaks.bed

In [55]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_DM6_DM7_merged_peaks.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_C14_merged_peaks.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/all_merged_peaks.bed

### Alternatively, merge beds of DM1_DM2d + Col-0 C12

In [2]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_broad_peaks.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_broad_peaks.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_and_C12_merged_peaks.bed

##### add "chr" to the first chromesome names

In [56]:
! awk '{print "chr"$0}' TOBIAS/merged_peaks/all_merged_peaks.bed > TOBIAS/merged_peaks/chr_all_merged_peaks.bed

In [3]:
! awk '{print "chr"$0}' TOBIAS/merged_peaks/DM1_DM2d_and_C12_merged_peaks.bed > TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed

### Annotation of peaks to nearest genes

#### download gtf from https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-58/gtf/arabidopsis_thaliana/Arabidopsis_thaliana.TAIR10.58.gtf.gz, then grep -v "^#" Arabidopsis_thaliana.TAIR10.58.gtf | awk '{print "chr"$0}' > Arabidopsis_thaliana.TAIR10.58_modified.gtf, use it for this

In [4]:
! uropa --bed TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --gtf /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58.gtf --show_attributes gene_id gene_name --feature_anchor start --distance 20000 10000 --feature gene -o ./TOBIAS/UROPA_DM1_DM2d/

2024-03-20 22:30:00 (3000412) [INFO]	Started UROPA 4.0.3
2024-03-20 22:30:00 (3000412) [INFO]	Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
2024-03-20 22:30:00 (3000412) [INFO]	Command-line call: /home/hu/miniconda3/envs/work2/bin/uropa --bed TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --gtf /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58.gtf --show_attributes gene_id gene_name --feature_anchor start --distance 20000 10000 --feature gene -o ./TOBIAS/UROPA_DM1_DM2d/
2024-03-20 22:30:00 (3000412) [INFO]	Reading configuration from commandline/input config
2024-03-20 22:30:00 (3000412) [INFO]	Preparing .gtf-file for fast access
[E::hts_idx_push] Unsorted positions on sequence #1: 29677904 followed by 26964087
2024-03-20 22:30:05 (3000412) [WARNING]	Indexing failed - the GTF is probably unsorted
2024-03-20 22:30:05 (3000412) [WARNING]	Attempting to sort with call: grep -v "^#" ./TOBIAS/UROPA_DM1_DM2d/chr_DM1_DM2d_and_C12_merged_pea

#### the file is chr_all_merged_peaks_finalhits.txt

In [18]:
! cut -f 1-6,16-17 TOBIAS/UROPA_DM1_DM2d/chr_DM1_DM2d_and_C12_merged_peaks_finalhits.txt | head -n 1 > TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated_header.txt

cut: write error: Broken pipe


In [17]:
! cut -f 1-6,16-17 TOBIAS/UROPA_DM1_DM2d/chr_DM1_DM2d_and_C12_merged_peaks_finalhits.txt | tail -n +2 > TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated.bed

#### ATACorrect

In [8]:
! TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --outdir ./TOBIAS/ATACorrect_C12 --cores 80

# TOBIAS 0.16.1 ATACorrect (run started 2024-03-20 22:35:01.943469)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --outdir ./TOBIAS/ATACorrect_C12 --cores 50

# ----- Input parameters -----
# bam:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam
# genome:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# peaks:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed
# regions_in:	None
# regions_out:	None
# blacklist:	None
# extend:	100
# split_strands:	False
# norm_off:	False
# track_off:	[]
# drop_chrom

In [9]:
! TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --outdir ./TOBIAS/ATACorrect_DM1_DM2d --cores 80

# TOBIAS 0.16.1 ATACorrect (run started 2024-03-20 22:43:49.612650)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --outdir ./TOBIAS/ATACorrect_DM1_DM2d --cores 50

# ----- Input parameters -----
# bam:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam
# genome:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# peaks:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed
# regions_in:	None
# regions_out:	None
# blacklist:	None
# extend:	100
# split_strands:	False
# norm_off:	False
# track_off:	

#### ScoreBigwig

In [10]:
! TOBIAS FootprintScores --signal TOBIAS/ATACorrect_C12/C12_merged_corrected.bw --regions TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --output TOBIAS/Footprints/C12_footprint.bw --cores 80

# TOBIAS 0.16.1 ScoreBigwig (run started 2024-03-20 22:53:19.520905)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FootprintScores --signal TOBIAS/ATACorrect_C12/C12_merged_corrected.bw --regions TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --output TOBIAS/Footprints/C12_footprint.bw --cores 80

# ----- Input parameters -----
# signal:	TOBIAS/ATACorrect_C12/C12_merged_corrected.bw
# output:	TOBIAS/Footprints/C12_footprint.bw
# regions:	TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed
# score:	footprint
# absolute:	False
# extend:	100
# smooth:	1
# min_limit:	None
# max_limit:	None
# fp_min:	20
# fp_max:	50
# flank_min:	10
# flank_max:	30
# window:	100
# cores:	80
# split:	100
# verbosity:	3


# ----- Output files -----
# TOBIAS/Footprints/C12_footprint.bw


2024-03-20 22:53:19 (3007218) [INFO]	Processing input files
2024-03-20 22:53:19 (3007218) [INFO]	- Opening input cutsite bigwig
2024-03-20 22:53:19 (3007218) [INFO]	- G

In [11]:
! TOBIAS FootprintScores --signal TOBIAS/ATACorrect_DM1_DM2d/DM1_DM2d_merged_corrected.bw --regions TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --output TOBIAS/Footprints/DM1_DM2d_foorprint.bw --cores 80

# TOBIAS 0.16.1 ScoreBigwig (run started 2024-03-20 22:55:06.293637)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FootprintScores --signal TOBIAS/ATACorrect_DM1_DM2d/DM1_DM2d_merged_corrected.bw --regions TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed --output TOBIAS/Footprints/DM1_DM2d_foorprint.bw --cores 80

# ----- Input parameters -----
# signal:	TOBIAS/ATACorrect_DM1_DM2d/DM1_DM2d_merged_corrected.bw
# output:	TOBIAS/Footprints/DM1_DM2d_foorprint.bw
# regions:	TOBIAS/merged_peaks/chr_DM1_DM2d_and_C12_merged_peaks.bed
# score:	footprint
# absolute:	False
# extend:	100
# smooth:	1
# min_limit:	None
# max_limit:	None
# fp_min:	20
# fp_max:	50
# flank_min:	10
# flank_max:	30
# window:	100
# cores:	80
# split:	100
# verbosity:	3


# ----- Output files -----
# TOBIAS/Footprints/DM1_DM2d_foorprint.bw


2024-03-20 22:55:06 (3008561) [INFO]	Processing input files
2024-03-20 22:55:06 (3008561) [INFO]	- Opening input cutsite bigwig
2024-

In [12]:
motifs = '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
fasta = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa'

#### BINDetect this command need to be run in a new env, use lower version of matplotlib

In [ ]:
! TOBIAS BINDetect --motifs ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --signals TOBIAS/Footprints/C12_footprint.bw TOBIAS/Footprints/DM1_DM2d_foorprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated.bed --peak_header TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output_DM1_DM2d --cond_names COL DM --cores 99

#### Visualize the difference in footprints between two conditions for all accessible sites:

### this!

In [19]:
! TOBIAS BINDetect --motifs ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --signals TOBIAS/Footprints/C12_footprint.bw TOBIAS/Footprints/DM1_DM2d_foorprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated.bed --peak_header TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output_DM1_DM2d_test1 --cond_names COL DM --cores 99

# TOBIAS 0.16.1 BINDetect (run started 2024-03-20 23:07:46.438919)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS BINDetect --motifs ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --signals TOBIAS/Footprints/C12_footprint.bw TOBIAS/Footprints/DM1_DM2d_foorprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated.bed --peak_header TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output_DM1_DM2d_test1 --cond_names COL DM --cores 99

# ----- Input parameters -----
# signals:	['TOBIAS/Footprints/C12_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_foorprint.bw']
# peaks:	TOBIAS/UROPA_DM1_DM2d/DM1_DM2d_and_C12_merged_peaks_annotated.bed
# motifs:	['./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme']
# genome:	/media/HDD2/donghui/diffTF_ath/diff

In [20]:
import subprocess
import os

def run_plot_heatmap(tf_name):
    # Define the output directory and ensure it exists
    output_dir = "TOBIAS/PlotHeatmap_DM1_DM2d"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the base path for BED files
    bed_base_path = f"TOBIAS/BINDetect_output_DM1_DM2d/{tf_name}/beds/{tf_name}"

    # # Define the BED files for bound and unbound in DM and COL conditions
    # bed_files = [
    #     f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",
    #       f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed"
    # ]

    # Define the command for PlotHeatmap
    cmd = [
        "TOBIAS", "PlotHeatmap", 
        "--TFBS"] + [ f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",] + [
        "--TFBS"] + [ f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed",] + [
        "--signals", "TOBIAS/ATACorrect_C12/C12_merged_corrected.bw", "TOBIAS/ATACorrect_DM1_DM2d/DM1_DM2d_merged_corrected.bw", 
        "--output", os.path.join(output_dir, f"{tf_name}_heatmap.pdf"),
        "--sort_by", "-2", '--title', tf_name
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotHeatmap for {tf_name} Success'
        print(f"PlotHeatmap for {tf_name} Success")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"

# Example usage
tf_name = "WRKY18_AT4G31800"
plot_heatmap_result = run_plot_heatmap(tf_name)



# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-20 23:14:50.729773)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_DM1_DM2d/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_DM1_DM2d/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_DM1_DM2d/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_DM1_DM2d/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_unbound.bed --signals TOBIAS/ATACorrect_C12/C12_merged_corrected.bw TOBIAS/ATACorrect_DM1_DM2d/DM1_DM2d_merged_corrected.bw --output TOBIAS/PlotHeatmap_DM1_DM2d/WRKY18_AT4G31800_heatmap.pdf --sort_by -2 --title WRKY18_AT4G31800

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_DM1_DM2d/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed', 'TOBIAS/BINDetect_output_DM1_DM2d/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed'], ['TOBIAS/BINDetect_output_DM1_DM

In [21]:
plot_heatmap_result = run_plot_heatmap('DOF4.7_AT4G38000')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-20 23:15:43.524584)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_DM1_DM2d/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed TOBIAS/BINDetect_output_DM1_DM2d/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_DM1_DM2d/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed TOBIAS/BINDetect_output_DM1_DM2d/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_unbound.bed --signals TOBIAS/ATACorrect_C12/C12_merged_corrected.bw TOBIAS/ATACorrect_DM1_DM2d/DM1_DM2d_merged_corrected.bw --output TOBIAS/PlotHeatmap_DM1_DM2d/DOF4.7_AT4G38000_heatmap.pdf --sort_by -2 --title DOF4.7_AT4G38000

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_DM1_DM2d/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed', 'TOBIAS/BINDetect_output_DM1_DM2d/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed'], ['TOBIAS/BINDetect_output_DM1_DM

In [125]:
! TOBIAS PlotHeatmap

__________________________________________________________________________________________

                                   TOBIAS ~ PlotHeatmap                                   
__________________________________________________________________________________________

PlotHeatmap plots a heatmap of signals from bigwig(s) (each row is one site) as well as
the aggregate signal across all sites.

------------------------------------------------------------------------------------------

Input / output arguments:
  --TFBS [ ...]           TFBS sites per column (*required)
  --signals [ ...]        Signals in bigwig format (*required)
  --output                Output filename (default: TOBIAS_heatmap.pdf)

Plot arguments:
  --plot-boundaries       Plot TFBS boundaries
  --share-colorbar        Share colorbar across all bigwigs (default: estimate colorbar
                          per bigwig)
  --flank 
  --title 
  --TFBS-labels [ ...]    Labels of TFBS (default: basename of --TFBS)
 

In [124]:
import matplotlib.pyplot as plt

# List all available colormaps
colormaps = plt.colormaps()
print(colormaps)


['magma', 'inferno', 'plasma', 'viridis', 'cividis', 'twilight', 'twilight_shifted', 'turbo', 'Blues', 'BrBG', 'BuGn', 'BuPu', 'CMRmap', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Spectral', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg', 'bwr', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar', 'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'jet', 'nipy_spectral', 'ocean', 'pink', 'prism', 'rainbow', 'seismic', 'spring', 'summer', 'terrain', 'winter', 'Accent', 'Dark2', 'Paired', 'Pastel1', 'Pastel2', 'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b', 'tab20c', 'magma_r', 'inferno_r', 'plasma_r', 'viridis_r', 'cividis_r', 'twilight_r', 'twilight_shifted_r', 'turbo_r', 'Blues_r', 'BrBG_r', 'BuGn_r', 'BuPu_r', 'CMRmap_r', 'GnBu_r', 'G

In [106]:
run_plot_heatmap('DOF4.7_AT4G38000')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:27.668495)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/DOF4.7_AT4G38000_heatmap.pdf --sort_by -2 --title DOF4.7_AT4G38000

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bou

'TOBIAS PlotHeatmap for DOF4.7_AT4G38000 Success'

In [107]:
run_plot_heatmap('WRKY46_AT2G46400')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:46.524501)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY46_AT2G46400_heatmap.pdf --sort_by -2 --title WRKY46_AT2G46400

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_DM_bou

'TOBIAS PlotHeatmap for WRKY46_AT2G46400 Success'

In [108]:
run_plot_heatmap('NAC058_AT3G18400')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:52.749742)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_bound.bed TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_DM_bound.bed TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/NAC058_AT3G18400_heatmap.pdf --sort_by -2 --title NAC058_AT3G18400

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_DM_bou

'TOBIAS PlotHeatmap for NAC058_AT3G18400 Success'

In [109]:
run_plot_heatmap('AT5G01380_AT5G01380')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:58.234722)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_bound.bed TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_DM_bound.bed TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/AT5G01380_AT5G01380_heatmap.pdf --sort_by -2 --title AT5G01380_AT5G01380

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/AT5

'TOBIAS PlotHeatmap for AT5G01380_AT5G01380 Success'

In [110]:
run_plot_heatmap('CDF5_AT1G69570')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:44:03.018867)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_bound.bed TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_DM_bound.bed TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/CDF5_AT1G69570_heatmap.pdf --sort_by -2 --title CDF5_AT1G69570

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_DM_bound.bed', 'TOBIAS/BINDetect_outpu

'TOBIAS PlotHeatmap for CDF5_AT1G69570 Success'

In [87]:
! TOBIAS PlotHeatmap 

__________________________________________________________________________________________

                                   TOBIAS ~ PlotHeatmap                                   
__________________________________________________________________________________________

PlotHeatmap plots a heatmap of signals from bigwig(s) (each row is one site) as well as
the aggregate signal across all sites.

------------------------------------------------------------------------------------------

Input / output arguments:
  --TFBS [ ...]           TFBS sites per column (*required)
  --signals [ ...]        Signals in bigwig format (*required)
  --output                Output filename (default: TOBIAS_heatmap.pdf)

Plot arguments:
  --plot-boundaries       Plot TFBS boundaries
  --share-colorbar        Share colorbar across all bigwigs (default: estimate colorbar
                          per bigwig)
  --flank 
  --title 
  --TFBS-labels [ ...]    Labels of TFBS (default: basename of --TFBS)
 

In [99]:
def run_plot_heatmap2(tf_name, width, height):
    # Define the output directory and ensure it exists
    output_dir = "TOBIAS/PlotHeatmap"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the base path for BED files
    bed_base_path = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}"

    # Define the command for PlotHeatmap
    cmd = [
        "python", "plot_heatmap2.py",
        "--TFBS", f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",
        "--TFBS", f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed",
        "--signals", "TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw", "TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw",
        "--output", os.path.join(output_dir, f"{tf_name}_heatmap.pdf"),
        "--sort_by", "-2",
        "--width", str(width),
        "--height", str(height),
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotHeatmap for {tf_name} Success'
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"
    
tf_name = "WRKY18_AT4G31800"
width = 3
height = 3
plot_heatmap_result = run_plot_heatmap2(tf_name, width, height)
plot_heatmap_result

'TOBIAS PlotHeatmap for WRKY18_AT4G31800 Success'

In [100]:
! python plot_heatmap2.py --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY18_AT4G31800_heatmap.pdf --sort_by -2 --width 3 --height 3

### CreateNetwork

In [3]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/*/beds/*_all.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_all


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-24 20:06:44.040306)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF1_AT1G49720/beds/ABF1_AT1G49720_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF2_AT1G45249/beds/ABF2_AT1G45249_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF3_AT4G34000/beds/ABF3_AT4G34000_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI3_AT3G24650/beds/ABI3_AT3G24650_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI4_AT2G40220/beds/ABI4_AT2G40220_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI5_AT2G36270/beds/ABI5_AT2G36270_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABO3_AT1G66600/beds/ABO3_AT1G66600_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_o

In [2]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/*/beds/*DM_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DM


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-24 20:16:23.248964)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF1_AT1G49720/beds/ABF1_AT1G49720_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF2_AT1G45249/beds/ABF2_AT1G45249_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF3_AT4G34000/beds/ABF3_AT4G34000_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI3_AT3G24650/beds/ABI3_AT3G24650_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI4_AT2G40220/beds/ABI4_AT2G40220_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI5_AT2G36270/beds/ABI5_AT2G36270_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABO3_AT1G66600/beds/ABO3_AT1G66600_DM_bound.bed /media/HDD2/donghui/bu

In [ ]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/*/beds/*COL_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_COL


#### Using all 600 TFs in the BINDetect is too much, use only DEG  l2f = 2

In [30]:
import pyreadr
import pandas as pd
import re

# Load the RDS file
result = pyreadr.read_r('../3DM_shoot/Rds_Rdata/Res_3DM.2023-04-21 14:05:06rds')

# Extract the DataFrame
df = result[None]

# Filter rows where 'contrast' column contains 'DM1/DM2d'
filtered_df = df[df['contrast'].str.contains('DM1/DM2d', na=False)]

# Select the 'AGI' and 'log2FoldChange' columns
df_DM1_DM2d = filtered_df[['AGI', 'log2FoldChange']]


In [31]:
#filter log2FoldChange abs > 2 
df_DM1_DM2d = df_DM1_DM2d[df_DM1_DM2d['log2FoldChange'].abs() > 2]
df_DM1_DM2d

,AGI,log2FoldChange
26369,AT1G30814,5.347096
26371,AT1G71695,-3.323255
26379,AT1G65630,-3.848987
26380,AT1G71390,3.127214
26384,AT1G73360,-3.777769
...,...,...
52724,ATCG00990,-2.809187
52725,ATCG01200,-4.638700
52727,ATCG01290,-2.920030
52728,ATCG00410,-2.612143


In [36]:
# Assuming df_DM1_DM2d is your DataFrame
agi_list = df_DM1_DM2d['AGI'].tolist()

# Join the elements with a space
agi_str = ' '.join(agi_list)

# Write to a text file
with open('./TOBIAS/meme/DEG_TFs.txt', 'w') as file:
    file.write(agi_str)

In [37]:
# ! echo 'AT3G56400' > ./TOBIAS/meme/DEG_TFs.txt
! TOBIAS FormatMotifs --input ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --output ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --filter ./TOBIAS/meme/DEG_TFs.txt

# TOBIAS 0.16.1 FormatMotifs (run started 2024-01-26 15:25:12.879348)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FormatMotifs --input ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --output ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --filter ./TOBIAS/meme/DEG_TFs.txt

# ----- Input parameters -----
# input:	['./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme']
# output:	./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar
# format:	jaspar
# task:	join
# filter:	./TOBIAS/meme/DEG_TFs.txt
# verbosity:	3


# ----- Output files -----
# ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar


2024-01-26 15:25:12 (1037664) [INFO]	Reading input files...
2024-01-26 15:25:12 (1037664) [INFO]	Read 619 motifs

2024-01-26 15:25:12 (1037664) [INFO]	Reading entries in ./TOBIAS/meme/DEG_TFs.txt
2024-01-26 15:25:12 (1037664) [INFO]	Read 8805 unique filter values
2024-01-26 15:25

In [ ]:
TOBIAS BINDetect --motifs ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --signals TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA/merged_peaks_annotated.bed --peak_header TOBIAS/UROPA/merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output_DEG --cond_names COL DM --cores 99

### CreateNetwork

In [ ]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/*/beds/*_all.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DEG_all


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-26 15:44:26.778598)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABI3_AT3G24650/beds/ABI3_AT3G24650_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABO3_AT1G66600/beds/ABO3_AT1G66600_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABR1_AT5G64750/beds/ABR1_AT5G64750_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AFO_AT2G45190/beds/AFO_AT2G45190_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AG_AT4G18960/beds/AG_AT4G18960_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL13_AT3G61120/beds/AGL13_AT3G61120_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL6_AT2G45650/beds/AGL6_AT2G45650_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetec

In [1]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/*/beds/*DM_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DEG_DM


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-26 15:36:29.735174)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABI3_AT3G24650/beds/ABI3_AT3G24650_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABO3_AT1G66600/beds/ABO3_AT1G66600_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABR1_AT5G64750/beds/ABR1_AT5G64750_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AFO_AT2G45190/beds/AFO_AT2G45190_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AG_AT4G18960/beds/AG_AT4G18960_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL13_AT3G61120/beds/AGL13_AT3G61120_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL6_AT2G45650/beds/AGL6_AT2G45650_DM_bound.bed /media/HDD2/donghui

In [ ]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/*/beds/*COL_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DEG_COL


#### TFBS

In [53]:
! TOBIAS TFBScan --motifs TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --fasta /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --cores 80 --outdir TOBIAS/TFBScan_DEG

# TOBIAS 0.16.1 TFBScan (run started 2024-01-30 16:51:19.789346)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS TFBScan --motifs TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --fasta /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --cores 80 --outdir TOBIAS/TFBScan_DEG

# ----- Input parameters -----
# motifs:	TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar
# fasta:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# regions:	None
# outdir:	TOBIAS/TFBScan_DEG
# outfile:	None
# naming:	name_id
# gc:	None
# pvalue:	0.0001
# keep_overlaps:	False
# add_region_columns:	False
# verbosity:	3
# split:	100
# cores:	80


2024-01-30 16:51:19 (1639287) [INFO]	Handling input files
2024-01-30 16:51:19 (1639287) [INFO]	Reading sequences from fasta
2024-01-30 16:51:19 (1639287) [STATS]	- Fo

In [56]:
! cat TOBIAS/TFBScan_DEG/*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed

In [59]:
! cat TOBIAS/TFBScan_DEG/WRKY*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.bed

In [67]:
! cat TOBIAS/TFBScan_DEG/DOF*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed

### ! TOBIAS PlotTracks

In [48]:
! echo -e "chr1\t12166281\t12169492" > ./TOBIAS/regions/ADR1.bed

In [45]:
$ TOBIAS PlotTracks --bigwigs test_data/*cell_corrected.bw --bigwigs test_data/*cell_footprints.bw --regions test_data/plot_regions.bed --sites test_data/binding_sites.bed --highlight test_data/binding_sites.bed --gtf test_data/genes.gtf --colors red darkblue red darkblue

SyntaxError: invalid syntax (578903902.py, line 1)

In [77]:
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_ADR1_all

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-31 09:57:25.514850)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_ADR1_all

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed
# highlight:	None
# gtf:	TOBIAS/gff3/Arabidopsis_thaliana.TAIR10

In [61]:
! echo -e "chr1\t12166281\t12169492" > ./TOBIAS/regions/ADR1.bed
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.bed --highlight TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-30 16:59:31.934206)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.bed --highlight TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.be

In [65]:
! echo -e "chr1\t12167781\t12168992" > ./TOBIAS/regions/ADR1.bed
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY70_AT3G56400.bed --highlight TOBIAS/TFBScan_DEG/WRKY70_AT3G56400.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY70

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-30 17:09:58.215590)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY70_AT3G56400.bed --highlight TOBIAS/TFBScan_DEG/WRKY70_AT3G56400.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY70

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/WRKY70_AT3G56400.bed
# highlight:	TOBIAS/

In [72]:
! echo -e "chr1\t12167781\t12168992" > ./TOBIAS/regions/ADR1.bed
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --highlight TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY18_AT4G31800

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-30 22:53:49.383880)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --highlight TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY18_AT4G31800

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed
# highligh

In [75]:
! echo -e "chr1\t12167981\t12168992" > ./TOBIAS/regions/ADR1.bed
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --highlight TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY18_AT4G31800

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-30 23:06:42.756823)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --highlight TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_WRKY18_AT4G31800

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/WRKY18_AT4G31800.bed
# highligh

In [71]:
! echo -e "chr1\t22387120\t22388600" > ./TOBIAS/regions/CIK1.bed
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/CIK1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_CIK1

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-30 22:11:25.673398)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/CIK1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_CIK1

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/CIK1.bed
# sites:	TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed
# highlight:	None
# gtf:	TOBIAS/gff3/Arabidopsis_thaliana.TA

In [76]:
! echo -e "chr1\t22387833\t22388600" > ./TOBIAS/regions/CIK1.bed
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/CIK1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_CIK1

# TOBIAS 0.16.1 PlotTracks (run started 2024-01-30 23:16:34.188450)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/CIK1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks_CIK1

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/CIK1.bed
# sites:	TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed
# highlight:	None
# gtf:	TOBIAS/gff3/Arabidopsis_thaliana.TA